目次：
1.リサイズ＆グレースケール
2.画像の水増し
3.画像を配列化(データリストの作成)
4.ラベルリストの作成
5.訓練データ・ラベル、テストデータ・ラベルをそれぞれシャッフルして保存(npyで保存)

grayscaler(resize=28,sample_num=2,used="train"):
resize：変更したいpixel数
sample_num：オリジナルが異なる画像の数
used：train,test,inputから選ぶ

In [48]:
def grayscaler(resize=28,sample_num=2,used="train"):
    import os
    from PIL import Image
    
    os.mkdir('./%s/gray_%d'%(used,resize))
    for j in range(1,sample_num+1):
        if j == 1:
            for i in range(10):
                img = './images_for_%s/%d.jpg'%(used,i)
                img = Image.open(img).convert('L')
                img_re = img.resize((resize,resize))
                img_re_arr = array(img_re)
                img = './%s/gray_%d/g_%d.jpg'%(used,resize,i)
                img = Image.fromarray(img_re_arr).save(img)
        else:
            #os.mkdir('./input/gray_%d'%(size))
            for i in range(10):
                img = './images_for_%s/%d (%d).jpg'%(used,i,j)
                img = Image.open(img).convert('L')
                img_re = img.resize((resize,resize))
                img_re_arr = array(img_re)
                img = './%s/gray_%d/g_%d (%d).jpg'%(used,resize,i,j)
                img = Image.fromarray(img_re_arr).save(img)
    print("Done")
#grayscaler(resize=50,sample_num=1,used="input")
grayscaler(resize=28,sample_num=2,used="test")

done


rsz_image(size=28,dataset_num=2,used="train",increment_roop=10000)
size：入力する画像のpixel数
dataset_num：オリジナルが違うデータセット毎に番号をつける
used：train,testから選ぶ
increment_roop：ImageDataGenerator.flowを何回ループさせるか(重複を防ぐためこの数字だけ複製されるわけではない)

In [5]:
def rsz_image(size=28,dataset_num=2,used="train",increment_roop=10000):
    import os
    from PIL import Image
    from numpy import array, shape, dtype, expand_dims
    import keras
    from keras.preprocessing.image import ImageDataGenerator
    
    datagen = ImageDataGenerator(rotation_range=180,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                 zoom_range=[0.8,1.2])
    datalist = []
    if dataset_num == 1:
        for i in range(10):
            img = './%s/gray_%d/g_%d.jpg'%(used,size,i)
            img = Image.open(img)
            img_arr = array(img)
            datalist.append(img_arr)
    else :
        for i in range(10):
            img = './%s/gray_%d/g_%d (%d).jpg'%(used,size,i,dataset_num)
            img = Image.open(img)
            img_arr = array(img)
            datalist.append(img_arr)
    datalist = array(datalist)
    datalist = expand_dims(datalist, axis=3) #(10,28,28,1)
    datalist = expand_dims(datalist, axis=1) #(10,1,28,28,1)
    print(datalist.shape)
    
    os.mkdir('./%s/rota_shif_zoom%d_%d'%(used,size,dataset_num))
    for n in range(10):
        print("Creating %d class"%(n+1))
        os.mkdir('./%s/rota_shif_zoom%d_%d/rsz_%d'%(used,size,dataset_num,n))
        a = datagen.flow(datalist[n],
                         batch_size=1,
                         shuffle=False,
                         save_to_dir='./%s/rota_shif_zoom%d_%d/rsz_%d'%(used,size,dataset_num,n),
                         save_prefix='rsz',
                         save_format='jpg')
        count = 0
        for batch in a:
            count += 1
            if count == increment_roop:
                break
    print("Done")
rsz_image(size=50,dataset_num=2,used="test",increment_roop=10000)

(10, 1, 50, 50, 1)
Creating 1 class
Creating 2 class
Creating 3 class
Creating 4 class
Creating 5 class
Creating 6 class
Creating 7 class
Creating 8 class
Creating 9 class
Creating 10 class
Done


画像データをnpyファイルで保存するための準備
lidirでファイルを読み込み、そこから5000ずつ配列にする。

create_datalist(size=28,sample_num=2,pick_out=5000,used="train")
size：入力する画像のpixel数
sample_num：オリジナルが異なる画像の数
pick_out：選び出してくる画像データの数 *ImageDataGenerator.flowとlistdirのランダム性に依存する
used：train,testから選ぶ
sample_num * pick_out が全データ数

In [6]:
def create_datalist(size=28,sample_num=2,pick_out=5000,used="train"):
    from PIL import Image
    from os import listdir
    from numpy import array,shape,expand_dims
    datalist = []
    for j in range(1,sample_num+1):
        if j == 1:
            print("sample1:")
            for i in range(10):
                print("image",i,end=" ")
                name = './%s/rota_shif_zoom%d_%d/rsz_%d/'%(used,size,j,i)
                for n in range(pick_out):
                    a = listdir(name)
                    img = Image.open(name+a[0])
                    img_arr = array(img)
                    datalist.append(img_arr)
        else:
            print("sample2:")
            for i in range(10):
                print("image",i,end=" ")
                name = './%s/rota_shif_zoom%d_%d/rsz_%d/'%(used,size,j,i)
                for n in range(pick_out):
                    a = listdir(name)
                    img = Image.open(name+a[0])
                    img_arr = array(img)
                    datalist.append(img_arr)
    datalist = array(datalist)
    datalist = expand_dims(datalist,axis=3)
    print(datalist.shape)
    print("Done")
    return datalist

#x_datalist = create_datalist(size=50,sample_num=2,pick_out=5000,used="train")
#y_datalist = create_datalist(size=50,sample_num=2,pick_out=2500,used="test")
#print(x_datalist.shape)

sample1:
image 0 image 1 image 2 image 3 image 4 image 5 image 6 image 7 image 8 image 9 sample2:
image 0 image 1 image 2 image 3 image 4 image 5 image 6 image 7 image 8 image 9 (50000, 50, 50, 1)
Done


配列化された画像の確認用関数(画素：黒0白255)
confirmation(size=28,input_data)
size=28：入力データのサイズ(pixel)
input_data：入力データ(三次元配列を読み込む)

In [12]:
def confirmation(size=28,input_data=x_datalist):
    for i in range(0, size):
        for j in range(0,size):
            if input_data[i, j] >= 127:
                print('*', end='')
            if input_data[i,j] < 127:
                print(' ',end='')
            if j ==(size-1):
                print()
confirmation(size=50,input_data=x_datalist[5000*0])
confirmation(size=50,input_data=x_datalist[5000*10])

              ***** *****                         
             *************                        
            ******  *******                       
           ******* ********                       
          ********  ********                      
         ********  **********                     
        ********    *********                     
       ********      ********                     
      ********        ********                    
     ********         *********                   
    ********           ********                   
   ********            ********                   
  ********              ********                  
 ********               *******                   
********                 *******                  
*******                   *****                   
******                   ******                   
*****                     * **   **               
****                            ***               
***                            

ラベルリストを作成
画像データの配列(データリスト)内は各温度につき0℃から5000ずつ順番に格納されている。
ラベルリストもそれに合わせる。

In [13]:
def create_labellist(dataset_num=2,pick_out=5000):
    from numpy import array,shape
    labellist = []
    for dataset in range(dataset_num):
        for label in range(10):
            for i in range(pick_out):
                labellist.append(label)
    labellist = array(labellist)
    print(labellist.shape)
    return labellist
        
x_labellist = create_labellist(dataset_num=2,pick_out=5000)
y_labellist = create_labellist(dataset_num=2,pick_out=2500)
print(x_labellist[5000*0])
print(x_labellist[5000*10])

(100000,)
(50000,)
0
0


データリストとラベルリストをシャッフルしてデータセットを作る
作るデータセットの内容は([訓練用画像],[訓練用ラベル],[テスト用画像],[テスト用ラベル])とする。
データセットをnpyファイルで保存する時は配列に変換されて保存される。そのため各データリストの要素数がすべて同じにならないように気をつける(ブロードキャストを避けるため)


In [14]:
def shuffle_dataset():
    from numpy import save
    from numpy.random import permutation
    a,b,c,d = x_datalist,x_labellist,y_datalist,y_labellist
    print(a.shape,b.shape,c.shape,d.shape)
    index1 = permutation(a.shape[0])
    index2 = permutation(c.shape[0])
    a,b = a[index1],b[index1]
    c,d = c[index2],d[index2]
    print(a.shape,b.shape,c.shape,d.shape)
    print("Test:",b[0],d[0])
    confirmation(size=50,input_data=a[0])
    print("-------------------------------------------------------------------------")
    confirmation(size=50,input_data=c[0])
    A = (a,b,c,d)
    save("x1e5y5e4_50_plan_b.npy",A) #convert array
shuffle_dataset()

(100000, 50, 50, 1) (100000,) (50000, 50, 50, 1) (50000,)
(100000, 50, 50, 1) (100000,) (50000, 50, 50, 1) (50000,)
Test: 1 5
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                                                  
                     ***                          
                  *****                           
                 ******                           
                 *******                          
                 *******                          
                 ******                           
       